<h1>Reddit Scraped Comments</h1>

<h3>Installation and import of libraries</h3>

In [40]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [41]:
folder_path = "/content/drive/My Drive/Reddit_Scraped_Comments/"
output_folder_path = "/content/drive/My Drive/"
file1 = "anti_lockdown_comments.csv"
file2 = "corona_lockdown_comments.csv"
file3 = "covid19_lockdown_comments.csv"

In [42]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

import nltk
nltk.download('stopwords') #download if don't have yet
from nltk.tokenize import word_tokenize, RegexpTokenizer # tokenize words
from nltk.corpus import stopwords

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<h3>Viewing Data</h3>

In [44]:
# Path
# folder_path = "../../covid-19-anti-lockdown-protests/Reddit_Scraped_Comments/"
# file1 = "anti_lockdown_comments.csv"
# file2 = "corona_lockdown_comments.csv"
# file3 = "covid19_lockdown_comments.csv"

In [45]:
anti_lockdown_comments_df = pd.read_csv(folder_path + file1)
anti_lockdown_comments_df

,Unnamed: 0,id,author,title,score,comms_num,created,timestamp,commenter,comment,top_lvl
0,0,ij071t,casualphilosopher1,"10,000 anti-lockdown protesters gather in Lond...",3039,600,1.598735e+09,2020-08-30 05:05:25,schu4KSU,People like this everywhere in the world. Dif...,1
1,0,ij071t,casualphilosopher1,"10,000 anti-lockdown protesters gather in Lond...",3039,600,1.598735e+09,2020-08-30 05:05:25,StupidizeMe,"Well, there's certainly enough of them to caus...",0
2,0,ij071t,casualphilosopher1,"10,000 anti-lockdown protesters gather in Lond...",3039,600,1.598735e+09,2020-08-30 05:05:25,Eltharion-the-Grim,They are largely absent from Asia. The only pl...,0
3,0,ij071t,casualphilosopher1,"10,000 anti-lockdown protesters gather in Lond...",3039,600,1.598735e+09,2020-08-30 05:05:25,Thisam,"Yup, the percentage of population who are easi...",0
4,0,ij071t,casualphilosopher1,"10,000 anti-lockdown protesters gather in Lond...",3039,600,1.598735e+09,2020-08-30 05:05:25,NaN,Short answer: Yes. Look at the USA and how Bre...,0
...,...,...,...,...,...,...,...,...,...,...,...
705,0,g4kdfs,vostok-Abdullah,Counter-Protesters in Scrubs Block Some Anti-L...,1483,192,1.587349e+09,2020-04-20 10:16:05,NaN,[removed],1
706,0,g4kdfs,vostok-Abdullah,Counter-Protesters in Scrubs Block Some Anti-L...,1483,192,1.587349e+09,2020-04-20 10:16:05,AutoModerator,Your comment has been removed because\n\n* **I...,0
707,0,g4kdfs,vostok-Abdullah,Counter-Protesters in Scrubs Block Some Anti-L...,1483,192,1.587349e+09,2020-04-20 10:16:05,Ameriican,They broke quarintine to tell others to not br...,1
708,0,g4kdfs,vostok-Abdullah,Counter-Protesters in Scrubs Block Some Anti-L...,1483,192,1.587349e+09,2020-04-20 10:16:05,AshingiiAshuaa,If the protesters' goal is to clog up streets ...,1


In [46]:
corona_lockdown_comments_df = pd.read_csv(folder_path + file2)
corona_lockdown_comments_df

,Unnamed: 0,id,author,title,score,comms_num,created,timestamp,commenter,comment,top_lvl
0,0,g1hpwu,Johari82,Ending coronavirus lockdowns will be a dangero...,6967,1484,1.586912e+09,2020-04-15 08:46:34,Skooter_McGaven,I wish we had an understanding of where the ma...,1
1,0,g1hpwu,Johari82,Ending coronavirus lockdowns will be a dangero...,6967,1484,1.586912e+09,2020-04-15 08:46:34,Richandler,> Is it overly family spread?\n\nI believe the...,0
2,0,g1hpwu,Johari82,Ending coronavirus lockdowns will be a dangero...,6967,1484,1.586912e+09,2020-04-15 08:46:34,NaN,This sounds horrible and I hate myself a littl...,0
3,0,g1hpwu,Johari82,Ending coronavirus lockdowns will be a dangero...,6967,1484,1.586912e+09,2020-04-15 08:46:34,lcbk,My husband and I are not yet confirmed to have...,0
4,0,g1hpwu,Johari82,Ending coronavirus lockdowns will be a dangero...,6967,1484,1.586912e+09,2020-04-15 08:46:34,ZombiGrn,In my neighborhood people started throwing par...,0
...,...,...,...,...,...,...,...,...,...,...,...
455,0,gr29as,frequenttimetraveler,Strict Physical Distancing May Be More Efficie...,909,352,1.590517e+09,2020-05-27 02:10:44,stillobsessed,A surprise lockdown period of 30 days wouldn't...,0
456,0,gr29as,frequenttimetraveler,Strict Physical Distancing May Be More Efficie...,909,352,1.590517e+09,2020-05-27 02:10:44,thisrockismyboone,Society would cease to function without infras...,0
457,0,gr29as,frequenttimetraveler,Strict Physical Distancing May Be More Efficie...,909,352,1.590517e+09,2020-05-27 02:10:44,SamH123,\- this depends a bit on what stage of spread ...,0
458,0,gr29as,frequenttimetraveler,Strict Physical Distancing May Be More Efficie...,909,352,1.590517e+09,2020-05-27 02:10:44,reini_urban,Strict physical distance may be statistically ...,1


<h3>Cleaning Data</h3>

In [47]:
# Drop columns with removed comments
anti_lockdown_comments_df = anti_lockdown_comments_df[anti_lockdown_comments_df.comment != "[removed]"]
corona_lockdown_comments_df = corona_lockdown_comments_df[corona_lockdown_comments_df.comment != "[removed]"]

In [48]:
list1 = anti_lockdown_comments_df["comment"].tolist()
list2 = corona_lockdown_comments_df["comment"].tolist()

In [49]:
def remove_special_chars(text):
    remove_chars = '[0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+'
    return re.sub(remove_chars, ' ', text)

def remove_digit_strings(text):
    return re.sub(r'\d+', '', text)

def remove_links(text):
    return re.sub(r'http\S+', '', text)

def clean_text(text):
    result_text = remove_links(text)
    result_text = remove_digit_strings(result_text)
    result_text = remove_special_chars(result_text)
    result_text = result_text.lower()
    return result_text

In [50]:
for i in range(len(list1)):
    list1[i] = clean_text(list1[i])

for i in range(len(list2)):
    list2[i] = clean_text(list2[i])

In [51]:
anti_lockdown_comments_df["cleaned_text"] = list1
corona_lockdown_comments_df["cleaned_text"] = list2

<ipython-input-51-aa79fcb2dad2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  anti_lockdown_comments_df["cleaned_text"] = list1
<ipython-input-51-aa79fcb2dad2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corona_lockdown_comments_df["cleaned_text"] = list2


In [52]:
titles_list1 = anti_lockdown_comments_df["title"].unique()
titles_list2 = corona_lockdown_comments_df["title"].unique()
# print(titles_list1)
# print(titles_list2)

title_comments = {}

for title in titles_list1:
  title_comments[title] = {}
  df = anti_lockdown_comments_df.loc[anti_lockdown_comments_df['title'] == title]
  title_comments[title]["text"] = df["cleaned_text"].tolist()

for title in titles_list2:
  title_comments[title] = {}
  df = corona_lockdown_comments_df.loc[corona_lockdown_comments_df['title'] == title]
  title_comments[title]["text"] = df["cleaned_text"].tolist()

<h3>Tokenizing Data</h3>

In [53]:
# Stopwords
stop_words = stopwords.words('english')
exclude_words = stop_words

#exclude common words 
exclude_words_extra = ["covid","lockdown","corona","pandemic","get","go","let","im","ive","would","one","also","to","say","day","well","month","thing"]

exclude_words.extend(exclude_words_extra)

In [54]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

for title in title_comments.keys():
  title_comments[title]["data_words"] = list(sent_to_words(title_comments[title]["text"]))
# data_words = list(sent_to_words(comments))

In [55]:
def bigram_trigram_models(title_key):
  data_words = title_comments[title_key]["data_words"]

  # Build the bigram and trigram models
  bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
  trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

  # Faster way to get a sentence clubbed as a trigram/bigram
  bigram_mod = gensim.models.phrases.Phraser(bigram)
  trigram_mod = gensim.models.phrases.Phraser(trigram)

  # # See trigram example
  # print(trigram_mod[bigram_mod[data_words[0]]])

  title_comments[title_key]["bigram"] = bigram
  title_comments[title_key]["trigram"] = trigram
  title_comments[title_key]["bigram_mod"] = bigram_mod
  title_comments[title_key]["trigram_mod"] = trigram_mod

for title in title_comments.keys():
  bigram_trigram_models(title)

print(title_comments)

{'10,000 anti-lockdown protesters gather in London to claim coronavirus is ‘a hoax’': {'text': ['people like this everywhere in the world   difference is  are there enough to gain political power ', 'well  there s certainly enough of them to cause a massive surge in covid  cases and deaths  and to take some of us with them ', 'they are largely absent from asia  the only place i recall reading about was a small group in japan  and the cults in s korea  \n\nthis is largely a western thing ', 'yup  the percentage of population who are easily deceived  fall for conspiracy theories  believe in whatever makes them feel warm and fuzzy instead of facts is about  \n\nthey used to be those crazy idiots who nobody ever listened to and recent political figures have united them  in several countries often with russian support ', 'short answer  yes  look at the usa and how brexit was passed for how  \n\nlong answer  you don t need to be the majority to gain power in a democracy you just need to depr

In [62]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in exclude_words] for doc in texts]

def make_bigrams(texts, bigram_mod):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts, bigram_mod, trigram_mod):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, nlp, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [64]:
def combined(title_key):
  data_words = title_comments[title_key]["data_words"]

  # Remove Stop Words
  data_words_nostops = remove_stopwords(data_words)
  data_words_nostops = data_words
  title_comments[title_key]["data_words_nostops"] = data_words_nostops 

  # Form Bigrams
  data_words_bigrams = make_bigrams(data_words_nostops, title_comments[title_key]["bigram_mod"])
  title_comments[title_key]["data_words_bigrams"] = data_words_bigrams

  # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
  # python3 -m spacy download en
  nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

  # Do lemmatization keeping only noun, adj, vb, adv
  data_lemmatized = lemmatization(data_words_bigrams, nlp, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
  title_comments[title_key]["data_lemmatized"] = data_lemmatized

  # print(data_lemmatized[:1])

for title in title_comments.keys():
  combined(title)

dict_keys(['text', 'data_words', 'bigram', 'trigram', 'bigram_mod', 'trigram_mod', 'data_words_nostops', 'data_words_bigrams', 'data_lemmatized'])


In [66]:
print(title_comments[list(title_comments.keys())[0]].keys())

dict_keys(['text', 'data_words', 'bigram', 'trigram', 'bigram_mod', 'trigram_mod', 'data_words_nostops', 'data_words_bigrams', 'data_lemmatized'])


In [68]:
def tokenize(title_key):
  data_lemmatized = title_comments[title_key]["data_lemmatized"]

  # Create Dictionary
  id2word = corpora.Dictionary(data_lemmatized)
  title_comments[title_key]["id2word"] = id2word

  # Create Corpus
  texts = data_lemmatized

  # Term Document Frequency
  corpus = [id2word.doc2bow(text) for text in texts]

  title_comments[title_key]["corpus"] = corpus

  # View
  # print(corpus[:1])

  # Human readable format of corpus (term-frequency)
  title_comments[title_key]["corpus_readable"] = [[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

for title in title_comments.keys():
  tokenize(title)

In [71]:
index = 1   # change this to see diff output
print(title_comments[list(title_comments.keys())[index]]["corpus_readable"])

[[('all', 1), ('charge', 2), ('die', 1), ('endangerment', 1), ('homicide', 1), ('misconduct', 1), ('people', 1), ('reckless', 1), ('result', 1)]]


<h3>LDA Model</h3>

In [72]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

# supporting function 2
def compute_perplexity_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    perplexity_score = lda_model.log_perplexity(corpus_sets[i])
    
    return perplexity_score

In [20]:
# import numpy as np
# import tqdm

# grid = {}
# grid['Validation_Set'] = {}
# # Topics range
# min_topics = 2
# max_topics = 11
# step_size = 1
# topics_range = range(min_topics, max_topics, step_size)
# # Alpha parameter
# alpha = list(np.arange(0.01, 1, 0.3))
# alpha.append('symmetric')
# alpha.append('asymmetric')
# # Beta parameter
# beta = list(np.arange(0.01, 1, 0.3))
# beta.append('symmetric')
# # Validation sets
# num_of_docs = len(corpus)
# corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
#                # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
#                # gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
#                corpus]
# corpus_title = ['100% Corpus']
# model_results = {'Validation_Set': [],
#                  'Topics': [],
#                  'Alpha': [],
#                  'Beta': [],
#                  'Coherence': [],
#                  'Perplexity': []
#                 }
# # Can take a long time to run
# if 1 == 1:
#     pbar = tqdm.tqdm(total=271)
    
#     # iterate through validation corpuses
#     for i in range(len(corpus_sets)):
#         # iterate through number of topics
#         for k in topics_range:
#             # iterate through alpha values
#             for a in alpha:
#                 # iterare through beta values
#                 for b in beta:
#                     # get the coherence score for the given parameters
#                     cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
#                                                   k=k, a=a, b=b)
#                     p = compute_perplexity_values(corpus=corpus_sets[i], dictionary=id2word, 
#                                                   k=k, a=a, b=b)
#                     # Save the model results
#                     model_results['Validation_Set'].append(corpus_title[i])
#                     model_results['Topics'].append(k)
#                     model_results['Alpha'].append(a)
#                     model_results['Beta'].append(b)
#                     model_results['Coherence'].append(cv)
#                     model_results['Perplexity'].append(p)
                    
#                     pbar.update(1)
#     pd.DataFrame(model_results).to_csv(output_folder_path + "reddit_lda_tuning_results-v2.csv", index=False)
#     pbar.close()

In [73]:
def lda_model(title_key):
  corpus = title_comments[title_key]["corpus"]
  id2word = title_comments[title_key]["id2word"]

  # Build LDA model
  num_topics = 4
  lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=num_topics, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)
  
  title_comments[title_key]["lda_model"] = lda_model
  
for title in title_comments.keys():
  lda_model(title)

In [74]:
index = 1   # change this to see diff output
print(title_comments[list(title_comments.keys())[index]]["lda_model"])

LdaModel<num_terms=846, num_topics=4, decay=0.5, chunksize=100>


In [76]:
def keyword_in_topics(title_key):
  corpus = title_comments[title_key]["corpus"]
  lda_model = title_comments[title_key]["lda_model"]

  # Print the Keyword in the 10 topics
  pprint(lda_model.print_topics())
  doc_lda = lda_model[corpus]
  title_comments[title_key]["doc_lda"] = doc_lda

for title in title_comments.keys():
  keyword_in_topics(title)

[(0,
  '0.029*"protest" + 0.025*"man" + 0.024*"crack" + 0.019*"get" + 0.017*"mask" '
  '+ 0.015*"death" + 0.014*"back" + 0.014*"protester" + 0.014*"time" + '
  '0.013*"black"'),
 (1,
  '0.019*"week" + 0.018*"hoax" + 0.017*"family" + 0.017*"people" + '
  '0.016*"wonder" + 0.009*"just" + 0.009*"make" + 0.009*"say" + 0.008*"more" + '
  '0.007*"conspiracy"'),
 (2,
  '0.023*"world" + 0.021*"so" + 0.020*"moron" + 0.020*"know" + 0.018*"hoax" + '
  '0.018*"think" + 0.017*"let" + 0.016*"disease" + 0.015*"fear" + '
  '0.013*"feed"'),
 (3,
  '0.027*"people" + 0.020*"just" + 0.016*"think" + 0.016*"stupidity" + '
  '0.015*"get" + 0.014*"american" + 0.012*"intelligent" + 0.012*"leak" + '
  '0.011*"go" + 0.011*"die"')]
[(0,
  '0.015*"comment" + 0.013*"subreddit" + 0.013*"coronavirus" + '
  '0.011*"discussion" + 0.011*"remove" + 0.011*"concern" + 0.010*"want" + '
  '0.009*"allow" + 0.009*"post" + 0.009*"political"'),
 (1,
  '0.018*"people" + 0.013*"more" + 0.012*"just" + 0.012*"nurse" + '
  '0.011*"in

In [77]:
index = 1   # change this to see diff output
print(title_comments[list(title_comments.keys())[index]]["doc_lda"])

In [79]:
def perplexity_coherence(title_key):
  corpus = title_comments[title_key]["corpus"]
  data_lemmatized = title_comments[title_key]["data_lemmatized"]
  id2word = title_comments[title_key]["id2word"]
  lda_model = title_comments[title_key]["lda_model"]

  # Compute Perplexity
  title_comments[title_key]["perplexity"] = lda_model.log_perplexity(corpus)
  # print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

  # Compute Coherence Score
  coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()

  title_comments[title_key]["coherence_model_lda"] = coherence_model_lda
  title_comments[title_key]["coherence_lda"] = coherence_lda

  # print('\nCoherence Score: ', coherence_lda)

for title in title_comments.keys():
  perplexity_coherence(title)

In [80]:
index = 1   # change this to see diff output
print(title_comments[list(title_comments.keys())[index]]["perplexity"])
print(title_comments[list(title_comments.keys())[index]]["coherence_lda"])

-6.932837000200108
0.5299064377495655


In [82]:
def optimal_model(title_key): 
  lda_model = title_comments[title_key]["lda_model"]

  optimal_model = lda_model
  model_topics = optimal_model.show_topics(formatted=False)

  title_comments[title_key]["model_topics"] = model_topics

for title in title_comments.keys():
  optimal_model(title)

In [84]:
index = 1   # change this to see diff output
print(title_comments[list(title_comments.keys())[index]]["model_topics"])

[(0, [('comment', 0.015130016), ('subreddit', 0.012740157), ('coronavirus', 0.012740157), ('discussion', 0.011380486), ('remove', 0.011380486), ('concern', 0.011379844), ('want', 0.009773806), ('allow', 0.008905278), ('post', 0.008635865), ('political', 0.008635865)]), (1, [('people', 0.018022068), ('more', 0.0128353555), ('just', 0.011825321), ('nurse', 0.011659771), ('infect', 0.011143839), ('let', 0.0105321165), ('so', 0.010481239), ('stupid', 0.010232635), ('child', 0.008301792), ('healthcare', 0.008258962)]), (2, [('protest', 0.02080872), ('people', 0.020408772), ('right', 0.014061776), ('just', 0.01384189), ('get', 0.010964323), ('think', 0.008228627), ('seriously', 0.008134189), ('hope', 0.007265694), ('wave', 0.0070253676), ('covid', 0.0066533373)]), (3, [('people', 0.04763542), ('get', 0.022073563), ('other', 0.021901375), ('right', 0.01811826), ('say', 0.010293005), ('traffic', 0.009393783), ('see', 0.008731382), ('water', 0.008515408), ('poor', 0.008515198), ('break', 0.0077

In [85]:
def explore_topic(lda_model, topic_number, topn, output=True):
    """
    accept a ldamodel, atopic number and topn vocabs of interest
    prints a formatted list of the topn terms
    """
    terms = []
    for term, frequency in lda_model.show_topic(topic_number, topn=topn):
        terms += [term]
        if output:
            print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))
    
    return terms

In [88]:
def explore_topic_by_title(title_key):
  lda_model = title_comments[title_key]["lda_model"]
  topic_summaries = []

  print(u'{:20} {}'.format(u'term', u'frequency') + u'\n')
  print(title_key)
  for i in range(4):
      print('Topic '+str(i)+' |---------------------\n')
      tmp = explore_topic(lda_model,topic_number=i, topn=10, output=True )
  #     print tmp[:5]
      topic_summaries += [tmp[:5]]
      # print

  title_comments[title_key]["topic_summaries"] = topic_summaries

for title in title_comments.keys():
  explore_topic_by_title(title)

term                 frequency

10,000 anti-lockdown protesters gather in London to claim coronavirus is ‘a hoax’
Topic 0 |---------------------

protest              0.029
man                  0.025
crack                0.024
get                  0.019
mask                 0.017
death                0.015
back                 0.014
protester            0.014
time                 0.014
black                0.013
Topic 1 |---------------------

week                 0.019
hoax                 0.018
family               0.017
people               0.017
wonder               0.016
just                 0.009
make                 0.009
say                  0.009
more                 0.008
conspiracy           0.007
Topic 2 |---------------------

world                0.023
so                   0.021
moron                0.020
know                 0.020
hoax                 0.018
think                0.018
let                  0.017
disease              0.016
fear                 0.015
feed   

In [100]:
import pyLDAvis.gensim_models as gensimvis
import pickle
import string

# Visualize the topics
for title in title_comments.keys():
  lda_model = title_comments[title]["lda_model"]
  corpus = title_comments[title]["corpus"]
  id2word = title_comments[title]["id2word"]

  file_title = title.translate(str.maketrans('', '', string.punctuation))
  file_title = file_title.replace(" ", "_")
  file_title = file_title[0:10]

  pyLDAvis.enable_notebook()
  vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
  LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
  pyLDAvis.save_html(LDAvis_prepared, "/content/drive/My Drive/viz_html/" + file_title +'.html')

  file_path = '/content/drive/My Drive/viz_html/' + file_title + '.html'
  with open(file_path, 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write("<h1>" + title.rstrip('\r\n') + "</h1>" + '\n' + content)

/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
